In [100]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


import time
import requests
import os
import pandas as pd

<h1> DataSet Building

In [122]:
genders= [ 'men','women']
categories= ['accessories','clothing','sports','perfumes','care','fashion','underwear']


In [123]:
def download_ads_by_topic(source_dir,gender,category,url,per_gender=True):

    
    driver = webdriver.Chrome()
    driver.maximize_window()  
    username = 'ravittt32@gmail.com'
    password = 'rKremer060422'
    driver.get('https://www.pinterest.com/')
    
    if per_gender == True:
        if category == 'general':
            search_term = f"{gender} advertisements" 
        else:
            search_term = f"{gender} {category} advertisements" 
    else: 
        search_term = f"{category} advertisements"  
    driver.get(url + search_term)
    
    save_dir = rf'{source_dir}\genders\{category}\{gender}'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    downloaded_images = set()
    
    num_scrolls = 50 # go down in the page to achive more photo
    
    for _ in range(num_scrolls):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(2) 
        
        images = driver.find_elements(By.TAG_NAME, 'img')
        current_images = {img.get_attribute('src') for img in images if img.get_attribute('src')}
    
        images = driver.find_elements(By.TAG_NAME, 'img')
        current_images = set()
        
        for img in images:
            try:
                src = img.get_attribute('src')
                if src:
                    current_images.add(src)
            except StaleElementReferenceException:
                continue
        
        new_images = current_images - downloaded_images
        for src in new_images:
            try:
                img_response = requests.get(src)
                filename = os.path.join(save_dir, src.split('/')[-1].split("?")[0])  # Split on "?" to remove URL parameters
                with open(filename, 'wb') as f:
                    f.write(img_response.content)
                downloaded_images.add(src)
            except requests.exceptions.RequestException as e:
                print("Failed to download an image:", e)
    
    # Close the browser
    driver.quit()
    
    print(f"Download completed. Images for {gender} in {category} field are saved in:", save_dir)


In [ ]:
#per gender
url = 'https://www.pinterest.com/search/pins/?q='
source_dir='ads_from_pinterest' 
for gender in genders:
    for category in categories:
        download_ads_by_topic(source_dir,gender,category,url)      

In [126]:
pinterest_data=pd.DataFrame(columns=['Ads','Target audience','field'])
source_dir = r'ads_from_pinterest\genders'
fields=os.listdir(source_dir)
genders=['men','women']
for field in fields:
    for gender in genders:
        for ads in os.listdir(os.path.join(source_dir,field,gender)):
            pinterest_data.loc[len(pinterest_data)]=[ads,gender,field]
pinterest_data

,Ads,Target audience,field
0,000cd0c2ec48e0e268f4f0a72e63db95.jpg,men,accessories
1,0019339fd71db7cc89506828681b07de.jpg,men,accessories
2,004ced1eb0349ab91d19afa9d659f2f7.jpg,men,accessories
3,01cf584a98888cd02513cd410a96eeff.jpg,men,accessories
4,0285a1f19dd89e5bd22a56cbda7c68db.jpg,men,accessories
...,...,...,...
3903,f8af3be9cd182368b4d7020b54eea05c.jpg,women,underwear
3904,f98c31d2f52809155aa146c443a59ae1.jpg,women,underwear
3905,fa4d5812b558a198a8e3192ba514c3e5.jpg,women,underwear
3906,fc7b3e0235b88c5a26766f2200128f65.jpg,women,underwear


In [127]:
pinterest_data.to_csv(r'ads_from_pinterest\pinterest_data.csv')

In [23]:
#per gender
neutral_categories=['Financial','Food','Healthy','Home Decor','Automobile','Baby Products']
url = 'https://www.pinterest.com/search/pins/?q='
source_dir='ads_from_pinterest' 
for category in neutral_categories:
        download_ads_by_topic(source_dir,'naturals',category,url,per_gender=False)      

Download completed. Images for naturals in Baby Products field are saved in: ads_from_pinterest\Baby Products\naturals


In [35]:
pinterest_data_ntrl=pd.DataFrame(columns=['Ads','Target audience','field'])
source_dir = r'ads_from_pinterest\naturals'
fields=os.listdir(source_dir)
for field in fields:
    for ads in os.listdir(os.path.join(source_dir,field)):
        pinterest_data_ntrl.loc[len(pinterest_data_ntrl)]=[ads,'natural',field]
pinterest_data_ntrl

,Ads,Target audience,field
0,0587b9de65bf129b34787288e595f6c2.jpg,natural,Automobile
1,0773ca58ea4d649b5f0234b53b6a87e5.jpg,natural,Automobile
2,0812d5afc259e1be82ab93df61ebfe0a.jpg,natural,Automobile
3,0ba81641135411f402d3f1cf4f2a56e2.jpg,natural,Automobile
4,0f8c9901fdb3ecff0c9bf37ee455b05a.jpg,natural,Automobile
...,...,...,...
662,f47aa274685a25b9451e27d52bc20e7a.jpg,natural,Home decor
663,f59e583d52fb2127031e3731810b692b.jpg,natural,Home decor
664,f6dfb1ec7ba763a896b9cf0dc3f56784.jpg,natural,Home decor
665,f73ee92d66e4f258fe2e64a0271b1265.jpg,natural,Home decor
